# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
csv_file = os.path.join('/Users/shampee/Desktop/Data_HW/monu-mel-data-pt-08-2021-u-c/02-Homework/06-Python-APIs/Instructions/output_data/city_weather.csv')

city_df = pd.read_csv(csv_file)
city_df


,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,3443061,Chui,UY,-33.6971,-53.4616,58.64,94,86,14.90
1,3366880,Hermanus,ZA,-34.4187,19.2345,52.02,66,9,5.86
2,3833367,Ushuaia,AR,-54.8000,-68.3000,42.46,87,90,9.22
3,3412093,Vestmannaeyjar,IS,63.4427,-20.2734,49.12,73,100,13.98
4,2110227,Butaritari,KI,3.0707,172.7902,82.98,71,86,15.03
...,...,...,...,...,...,...,...,...,...
535,2120612,Tigil',RU,57.8000,158.6667,49.42,83,100,8.23
536,1695546,Pandan,PH,14.0453,124.1694,80.98,83,100,5.88
537,1605102,Wang Saphung,TH,17.3010,101.7685,75.22,94,90,3.44
538,1254390,Tiruchchendur,IN,8.4833,78.1167,81.55,75,88,6.64


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)

In [8]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
ideal_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
30,2214432,Nālūt,LY,30.3333,10.8500,82.76,18,0,6.91
47,3393536,Olinda,BR,-8.0089,-34.8553,76.73,61,0,6.91
58,286245,Sur,OM,22.5667,59.5289,84.33,67,1,9.51
68,5546220,Saint George,US,37.1041,-113.5841,84.79,14,1,0.00
140,5146092,Ashtabula,US,41.7001,-80.7665,78.03,69,1,4.61
163,88319,Benghazi,LY,32.1167,20.0667,83.32,44,0,6.91
194,5354943,Half Moon Bay,US,37.4636,-122.4286,83.79,27,1,0.00
248,4116834,Jonesboro,US,35.8423,-90.7043,86.83,55,1,6.91
257,5364271,Laguna,US,38.4210,-121.4238,88.65,17,1,3.44
263,88533,Awjilah,LY,29.1081,21.2869,85.86,22,0,8.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
30,2214432,Nālūt,LY,30.3333,10.8500,82.76,18,0,6.91,
47,3393536,Olinda,BR,-8.0089,-34.8553,76.73,61,0,6.91,
58,286245,Sur,OM,22.5667,59.5289,84.33,67,1,9.51,
68,5546220,Saint George,US,37.1041,-113.5841,84.79,14,1,0.00,
140,5146092,Ashtabula,US,41.7001,-80.7665,78.03,69,1,4.61,
163,88319,Benghazi,LY,32.1167,20.0667,83.32,44,0,6.91,
194,5354943,Half Moon Bay,US,37.4636,-122.4286,83.79,27,1,0.00,
248,4116834,Jonesboro,US,35.8423,-90.7043,86.83,55,1,6.91,
257,5364271,Laguna,US,38.4210,-121.4238,88.65,17,1,3.44,
263,88533,Awjilah,LY,29.1081,21.2869,85.86,22,0,8.01,


In [11]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df        

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
30,2214432,Nālūt,LY,30.3333,10.8500,82.76,18,0,6.91,NaN
47,3393536,Olinda,BR,-8.0089,-34.8553,76.73,61,0,6.91,Hotel 7 Colinas
58,286245,Sur,OM,22.5667,59.5289,84.33,67,1,9.51,Sur Plaza Hotel
68,5546220,Saint George,US,37.1041,-113.5841,84.79,14,1,0.00,Best Western Coral Hills
140,5146092,Ashtabula,US,41.7001,-80.7665,78.03,69,1,4.61,NaN
163,88319,Benghazi,LY,32.1167,20.0667,83.32,44,0,6.91,Tibesti Hotel
194,5354943,Half Moon Bay,US,37.4636,-122.4286,83.79,27,1,0.00,Oceano Hotel & Spa Half Moon Bay Harbor
248,4116834,Jonesboro,US,35.8423,-90.7043,86.83,55,1,6.91,Hampton Inn Jonesboro
257,5364271,Laguna,US,38.4210,-121.4238,88.65,17,1,3.44,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
263,88533,Awjilah,LY,29.1081,21.2869,85.86,22,0,8.01,NaN


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))